# Overall Energy Demand Notebook: Exploring how a homes overall energy demand is attributable to at-home EV charging

<p>We will be using Pecan Street Inc. data from dataport to calculate how much overall energy demand is used in homes by electric vehicle charging.<br><br>
Pecans Streets data can be obtained by applying for a dataport account at https://www.dataport.pecanstreet.org.</p>

In [15]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
import numpy as np
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

'/usr/bin/python3.8'

In [16]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("./config/config.txt")

In [17]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))

In [18]:
#Select a list of Texas homes from dataport metadata having CAR and also has data for year 2018.
query = """select distinct dataid from other_datasets.metadata 
                                          where car1='yes' and grid='yes'
                                          and egauge_1min_min_time < '2018-01-01' 
                                          and egauge_1min_max_time > '2019-01-01'
                                          and state='Texas'
                                          and (egauge_1min_data_availability like '100%' 
                                               or 
                                               egauge_1min_data_availability like '99%');
         """

df = pd.read_sql_query(sqla.text(query), engine)

In [19]:
# grab dataids and convert them to a string to put into the SQL query
dataids_list = df['dataid'].tolist()
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

28 dataids selected listed here:


[114,
 379,
 1169,
 1202,
 1354,
 2814,
 3368,
 3829,
 3967,
 5109,
 5357,
 5450,
 5749,
 6139,
 6248,
 6691,
 7024,
 7850,
 7989,
 8142,
 8645,
 8847,
 8857,
 9484,
 9609,
 9647,
 9776,
 9932]

In [20]:
#Check data completeness for dataids selected from metadata above.

query2 = """select dataid,count(*) total_rec from electricity.eg_realpower_1min 
            where dataid in ({})""".format(dataids_str)
query2 = query2 + """ and localminute >= '2018-01-01' and localminute < '2019-01-01' group by 1"""

df2 = pd.read_sql_query(sqla.text(query2), engine)

In [21]:
#Select homes with atleast 99% data availability for year 2018.
df2['perc'] = (df2['total_rec']/525600)*100
final_dataids = df2[df2['perc'] >= 99]
final_dataids['dataid'].count()

27

In [22]:
# Pull data for homes
final_dataids_list = final_dataids['dataid'].tolist()
print("{} dataids selected listed here:".format(len(final_dataids_list)))
final_dataids_str = ','.join(list(map(str, final_dataids_list)))
final_dataids_str
final_dataids_list

27 dataids selected listed here:


[114,
 379,
 1169,
 1202,
 1354,
 2814,
 3368,
 3829,
 3967,
 5109,
 5357,
 5450,
 5749,
 6139,
 6248,
 6691,
 7024,
 7850,
 7989,
 8142,
 8645,
 8847,
 9484,
 9609,
 9647,
 9776,
 9932]

In [37]:

data_pull = """select localminute::timestamp,car1,grid,solar 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-03-01' and localminute <  '2018-06-01' """
data_pull = data_pull + """AND dataid in ({})""".format(final_dataids_str)

data_df = pd.read_sql_query(sqla.text(data_pull), engine)

In [41]:
#replace null with 0's
day_df = data_df.fillna(0)
# convert localminute to pandas daytime type
day_df['datetime'] = pd.to_datetime(day_df['localminute'])

day_df = day_df.set_index('datetime')

#create day column
day_df['day'] = day_df.index.day

#create new dataframes with grid, solar, car1, and day column
day_df_new = pd.DataFrame(day_df, columns = ['grid', 'solar', 'car1', 'day'])

# group data based on day and get avg
day_df_grouped = day_df_new.groupby(['day']).mean()

day_df_grouped['demand'] = day_df_grouped['car1'] / (day_df_grouped['grid']+ day_df_grouped['solar']) * 100

day_df_grouped.head(10)


,grid,solar,car1,demand
day,,,,
1,0.415982,0.591499,0.141586,14.053443
2,0.448659,0.611549,0.140052,13.209873
3,0.644773,0.392469,0.134011,12.919935
4,0.492557,0.496827,0.156142,15.781713
5,0.242824,0.783674,0.154365,15.038017
6,0.233964,0.898189,0.163874,14.474548
7,0.308313,0.795541,0.141104,12.782870
8,0.272400,0.902010,0.156608,13.335014
9,0.434916,0.615579,0.144223,13.729029


In [42]:
print(day_df_grouped['demand'].mean())

12.099395212185625


In [43]:
#replace null with 0's
week_df = data_df.fillna(0)
# convert localminute to pandas daytime type
week_df['datetime'] = pd.to_datetime(week_df['localminute'])

week_df = week_df.set_index('datetime')

#create day column
week_df['week'] = week_df.index.week

#create new dataframes with grid, car1, and week column
week_df_new = pd.DataFrame(week_df, columns = ['grid', 'solar', 'car1', 'week'])

# group data based on week and get avg
week_df_grouped = week_df_new.groupby(['week']).mean()

week_df_grouped['demand'] = week_df_grouped['car1'] / (week_df_grouped['grid']+week_df_grouped['solar']) * 100

week_df_grouped.head(10)

<ipython-input-43-8f61b09a3c12>:9: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  week_df['week'] = week_df.index.week


,grid,solar,car1,demand
week,,,,
9,0.314949,0.618052,0.123255,13.210628
10,0.239347,0.745600,0.149813,15.210260
11,0.248457,0.805987,0.133309,12.642585
12,0.262556,0.748703,0.108095,10.689190
13,0.271283,0.725321,0.116681,11.707881
14,0.410976,0.613284,0.162334,15.848886
15,0.122479,0.876475,0.159171,15.933771
16,0.126625,0.876414,0.141265,14.083685
17,0.140430,0.965652,0.145467,13.151555


In [45]:
print(week_df_grouped['demand'].mean())

12.362583361293526


# Observations:
As seen above daily and weekly overall demand of EV chargers stays pretty consistant. On average both daily and week demand is a little over 12%. At most demand peaks at about 16% and at min deman peaks at 10%.